###    Задача - решить систему линейных уравнений, используя три метода: прямой, итерационный и вариационный.
В качестве прямого метода выбран метод Гаусса, в качестве итерационного - метод Якоби, 
в качестве вариационного - метод невязок.

In [1]:
import numpy as np

#### Поле со входными полями. 
A_start*x = b_start - система, котурую нужно решить. 
<br> number_of_iterations - ограничение сверху на количество итерациий при решении итерационным и вариационным методами

In [2]:
number_of_iterations = 150

A_start = np.array([[11.0, 2.0, 3.0],
                    [4.0, 15.0, 6.0],
                    [7.0, 8.0, 21.0]])
b_start = np.array([10.0, 11.0, 12.0])

## Метод Гаусса

Складываем строки таким образом, чтобы получить верхнюю треугольную матрицу. На каждом шаге к i+1 строке добавляется i-ая, <br>домноженная на -A[i+1][i]/A[i][i]. Далее решаем треугольную матрицу следующим образом: на каждом шаге находит $x_i = \frac{b_i}{A_{ii}}$, вычитает из b i-ый столбец матрицы А, домноженный на $x_i$

#### вспомогательные функции: 
add_line - добавляет строку с номером i_add, домноженную на k к строке i_target системы
<br> exchange_lines - меняет i и j строки матрицы местами
<br> find_nonzero_j - находит строку, в которой j элемент не равен 0
<br> solve_trianjular_system - решает систему с верхнетреугольной матрицей

In [3]:
def add_line(i_target, i_add, k=1):
    # add [i_add] line of system, multiplied by k, to [i_target] line
    A[i_target] += k*A[i_add]
    b[i_target] += k*b[i_add]


def exchange_lines(i, j):
    if i != j:
        c = np.copy(A[i])
        A[i] = A[j]
        A[j] = c


def find_nonzero_j(i, j):
    # returns number of line after i (i line included) with nonzero j element
    for k in range(A.shape[0])[i:]:
        if A[k][j] != 0:
            return k
        else:
            print "zero column detected"


def solve_triangular_system(A, b):
    # solve system with top-triangular matrix
    solution = list()
    for i in range(A.shape[0]-1, -1, -1):
        solution.append(b[i]/A[i][i])
        for k in range(i):
            b[k] -= A[k][i]*solution[-1]
    solution.reverse()
    return solution

#### реализация метода Гаусса:
переменная direct_solution будет содержать точное решение системы 

In [4]:
A = np.copy(A_start)
b = np.copy(b_start)

for i in range(A.shape[0]-1):
    exchange_lines(i, find_nonzero_j(i, i))
    map(lambda j: add_line(j, i, -1*A[j][i]/A[i][i]), range(A.shape[0])[i+1:])

direct_solution = solve_triangular_system(A, b)
print "direct method (Gauss):" + str(direct_solution)

direct method (Gauss):[0.78815489749430523, 0.47152619589977213, 0.12908124525436601]


## Метод Якоби
Проверяем матрицу системы на условие диагонального преобладания (достаточное условие сходимости метода Якоби): модуль каждого элемента на диагонали должен быть больше суммы всех остальных элементов в строке. Если уловие не выполнено, решать не пытаемся.
<br><br>Далее аскладываем исходную матрицу А в сумму матриц A = L + D + U, где L и U - нижняя и верхняя треугольные матрицы с нулями на диагонали, D - диагональная матрица. Применяя рекурентную формулу $x_{n+1} = -D^{-1}(L+U)x_n + D^{-1}b$, проводим number_of_itterations иттераций.
<br>начальное приближение - нулевой вектор

#### вспомогательные функции: 
check_covergence - проверяет условие диагонального преобладания
<br> reverse_matrix - считает для диагональной матрицы обратную

In [5]:
def check_covergence(matrix):
    for i in range(matrix.shape[0]):
        summ = 0
        for j in range(matrix.shape[0]):
            if j!=i:
                summ += matrix[i][j]
        if np.abs(matrix[i][i] <= summ):
            print "condition for covergence is not sutisfied"
            return False
    return True


def reverse_matrix(matrix):
    # calculates revers matrix to diagonal matrix
    reversed_matrix = np.copy(matrix)
    for i in range(matrix.shape[0]):
        reversed_matrix[i][i] = 1/reversed_matrix[i][i]
    return reversed_matrix

#### реализация метода Якоби:
Проверяем достаточное условие сходимости, если выполнено, разбиваем матрицу на указанную сумму и делаем number_of_itterations шагов
<br>Выводим решение, число проделанных шагов и точность (норма разности полученного решения с точным)

In [6]:
A = np.copy(A_start)
b = np.copy(b_start)
x = np.zeros(A.shape[0])

if check_covergence(A) :
    L, D, U = np.zeros(A.shape), np.zeros(A.shape), np.zeros(A.shape)
    for i in range(A.shape[0]):
        for j in range(A.shape[0]):
            if i == j:
                D[i][j] = A[i][j]
            elif i > j:
                L[i][j] = A[i][j]
            elif i<j:
                U[i][j] = A[i][j]
                
    R = -1*np.dot(reverse_matrix(D), L+U)
    F = np.dot(reverse_matrix(D), b)
    x = np.zeros(A.shape[0])

    for i in range(number_of_iterations):
        x = np.dot(R, x) + F

    print "iterative method (Jacobi): " + str(x) + ", number of iterations: " + str(number_of_iterations) + ", accuracy = " + str(np.linalg.norm(direct_solution - x))


iterative method (Jacobi): [ 0.7881549   0.4715262   0.12908125], number of iterations: 150, accuracy = 1.27192026216e-16


## Метод невязок

начальное приближение - нулевой вектор
<br>рекурентная формула для определения следующих приближений:
<br>$x_{n+1} = x_n + \frac{(Ar, r)}{(Ar, Ar)}r$,  <br>где $r = b - Ax_n$ - невязка
<br><br> Выводит решение, количество проделанных итераций (в какой-то момент невязка может становиться нулевой в пределах машинной точности, на этом моменте выходим из цикла) и точность (норма разности решения с точным) 

In [7]:
A = np.copy(A_start)
b = np.copy(b_start)
x = np.zeros(A.shape[0])

for i in range(number_of_iterations):
    r = b - np.dot(A, x)
    if (np.linalg.norm(r) == 0):
        number_of_iterations = i
        break
    x = x + np.dot(np.dot(A, r), r)/np.linalg.norm(np.dot(A, r))**2*r

print "variation method (residual): " + str(x) + ", number of iterations: " + str(
    number_of_iterations) + ", accuracy = " + str(np.linalg.norm(direct_solution - x))

variation method (residual): [ 0.7881549   0.4715262   0.12908125], number of iterations: 31, accuracy = 6.20633538312e-17
